In [152]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import sys  
import joblib
sys.path.insert(0, './myCode')
from PLModel import PLModel
from utils import loadModelCheckpoint

In [3]:
from efficientnet_pytorch import EfficientNet

# Define image size per model (as you provided)
imageSizePerCNN = {
    "Resnext50": 600,
    "Resnet152": 600,
    "EfficientNetB7": 600,
    "EfficientNetB6": 528,
    "EfficientNetB5": 456,
    "EfficientNetB4": 380,
}

In [80]:
# The final order in which we want to feed the (averaged) logits into the meta-classifier
ARCHITECTURE_ORDER = [
    "EfficientNetB4",
    "EfficientNetB5",
    "EfficientNetB6",
    "Resnext50",
    "Resnet152"
]

In [175]:
# Define the class labels
CLASS_LABELS = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

class SkinDiseaseClassifier:
    """
    A class that loads multiple CNNs from a given folder structure and
    runs inference on a skin disease image. Each model has its own image-size
    requirement and possibly different architecture.
    """

    def __init__(
        self,
        base_folder: str, 
        metaclassifier_name: str, 
        device: str = 'cpu'
    ):
        """
        :param base_folder: The path to the main folder containing subfolders for each model.
                            Each subfolder should have a 'weights' directory with at least one .pth file.
        :param device: The torch device to use ('cpu' or 'cuda').
        """
        self.device = device
        self.base_folder = base_folder

        # Load the scikit-learn meta-classifier from 'metaclassifier' folder
        self.meta_model = self._load_meta_classifier(metaclassifier_name)
        # models_dict will map 'folder_name' -> {'model': model, 'transform': transform}
        self.subfolder_models = self._load_all_models()

    def _load_all_models(self):
        """
        Iterates subfolders in `base_folder` and loads any that are not "metaclassifier".
        Returns a dict:
            {
              "Resnext50_0Fold": ( "Resnext50", model, transform ),
              "Resnext50_1Fold": ( "Resnext50", model, transform ),
              "EfficientNetB6_3Fold": ( "EfficientNetB6", model, transform ),
              ...
            }
        """
        subfolder_dict = {}
        for entry in os.listdir(self.base_folder):
            # skip the meta-classifier folder
            if entry.lower() == "metaclassifier":
                continue

            model_subfolder = os.path.join(self.base_folder, entry)
            if os.path.isdir(model_subfolder):
                ckpt_files = [f for f in os.listdir(model_subfolder) if f.endswith('.ckpt')]
                if not ckpt_files:
                    print(f"No .pth file found in {model_subfolder}, skipping.")
                    continue
                weight_path = os.path.join(model_subfolder, ckpt_files[0])

                # Identify architecture (e.g. "Resnext50") from folder name
                arch_name = self._parse_base_model_name(entry)
                # Build model + transform
                model, transform = self._create_model_and_transform(arch_name, weight_path)

                subfolder_dict[entry] = {
                    "name" : arch_name, 
                    "model" : model, 
                    "transform" : transform
                }
        return subfolder_dict

    def _parse_base_model_name(self, folder_name: str) -> str:
        """
        E.g. "Resnext50_0Fold" -> "Resnext50".
             "EfficientNetB6_3Fold" -> "EfficientNetB6".
        We'll do a series of if/elif checks to match your naming.
        """
        name_lower = folder_name.lower()
        if 'efficientnetb4' in name_lower:
            return "EfficientNetB4"
        elif 'efficientnetb5' in name_lower:
            return "EfficientNetB5"
        elif 'efficientnetb6' in name_lower:
            return "EfficientNetB6"
        elif 'resnext' in name_lower:
            return "Resnext50"
        elif 'resnet152' in name_lower:
            return "Resnet152"
        else:
            raise ValueError(f"Unknown model type in folder name: {folder_name}")

    def _load_meta_classifier(
        self,
        metaclassifier_name
    ):
        """
        Load the scikit-learn meta-classifier from the 'metaclassifier' subfolder.
        For example, if you saved it as model.pkl or meta_model.pkl.
        """
        # Adjust the filename as necessary:
        meta_classifier_path = os.path.join('metaclassifiers', f'{metaclassifier_name}.pkl')
        if not os.path.exists(meta_classifier_path):
            raise FileNotFoundError(f"Could not find meta-classifier at: {meta_classifier_path}")
        
        meta_model = joblib.load(meta_classifier_path)
        return meta_model

    def _create_model_and_transform(self, folder_name: str, weight_path: str):
        """
        1) Parse the folder name to identify the base model (e.g., 'EfficientNetB4', 'Resnext50', etc.).
        2) Create and load that model, along with a transform that resizes to the required input dimension.
        3) Return a dict containing the model and the transform.
        """
        base_model_name = self._parse_base_model_name(folder_name)
        if base_model_name not in imageSizePerCNN:
            raise ValueError(f"Unrecognized base model in '{folder_name}'. "
                             f"Expected one of {list(imageSizePerCNN.keys())}, "
                             f"but got '{base_model_name}'.")

        # Get required image size
        image_size = imageSizePerCNN[base_model_name]

        # Define the transform for this specific model
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ])

        # Create and load the model
        model = self._load_model_architecture(base_model_name)
        state_dict = torch.load(weight_path, map_location=self.device)
        model = loadModelCheckpoint(weight_path, model)
        model = model.to(self.device)
        model.eval()

        return model, transform

    def _parse_base_model_name(self, folder_name: str) -> str:
        """
        Given a subfolder name like 'EfficientNetB4_0Fold' or 'Resnext_4Fold',
        extract the correct key from 'imageSizePerCNN'.
        We'll do a simple series of 'if' checks for known possibilities.
        Adjust as needed if your naming is more complex.
        """
        folder_name_lower = folder_name.lower()

        if 'efficientnetb7' in folder_name_lower:
            return 'EfficientNetB7'
        elif 'efficientnetb6' in folder_name_lower:
            return 'EfficientNetB6'
        elif 'efficientnetb5' in folder_name_lower:
            return 'EfficientNetB5'
        elif 'efficientnetb4' in folder_name_lower:
            return 'EfficientNetB4'
        elif 'resnext' in folder_name_lower:
            # For 'Resnext_4Fold', let's assume it means 'Resnext50'
            return 'Resnext50'
        elif 'resnet152' in folder_name_lower:
            return 'Resnet152'
        else:
            # If none matched, you can handle it or raise an error
            raise ValueError(f"Could not parse a known model name from: {folder_name}")

    def _load_model_architecture(self, base_model_name: str):
        """
        Create a model for the given base_model_name using torch.hub or efficientnet_pytorch.
        We assume 8 output classes. Modify as needed.
        """
        if base_model_name == "Resnext50":
            model = torch.hub.load(
                'pytorch/vision:v0.9.0',
                'resnext50_32x4d',
                pretrained=True
            )
            num_f = model.fc.in_features
            model.fc = nn.Linear(num_f, 8)

        elif base_model_name == "Resnet152":
            model = torch.hub.load(
                'pytorch/vision:v0.9.0',
                'resnet152',
                pretrained=True
            )
            num_f = model.fc.in_features
            model.fc = nn.Linear(num_f, 8)

        elif base_model_name == "EfficientNetB6":
            # from efficientnet_pytorch import EfficientNet
            model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=8)

        elif base_model_name == "EfficientNetB7":
            model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=8)

        elif base_model_name == "EfficientNetB5":
            model = EfficientNet.from_pretrained('efficientnet-b5', num_classes=8)

        elif base_model_name == "EfficientNetB4":
            model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=8)

        else:
            raise ValueError(f"No architecture defined for {base_model_name}.")

        model = PLModel('base_model_name', model)

        return model

    def predict(self, image_path: str):
        """
        Runs inference for each subfolder model, but then aggregates (averages) *logits* by architecture.
        
        Returns a dictionary keyed by architecture name, each containing:
        {
           "aggregated_logits": [8 floats],
           "aggregated_probabilities": [8 floats],
           "predicted_class": str
        }
        """
        # 1) Group subfolders by architecture
        arch_to_subfolders = {}
        for subfolder_name, model_obj in self.subfolder_models.items():
            arch_to_subfolders.setdefault(model_obj["name"], []).append(subfolder_name)

        # 2) Open the image once
        original_image = Image.open(image_path).convert('RGB')

        # We'll store final predictions in a dict keyed by architecture
        final_predictions = {}

        # 3) For each architecture, gather & average the logits from all folds
        for arch_name, subfolders in arch_to_subfolders.items():
            logits_list = []

            # For each fold subfolder, run inference & store logits
            for sf in subfolders:
                model = self.subfolder_models[sf]["model"]
                transform = self.subfolder_models[sf]["transform"]

                # Transform the image, run model
                input_tensor = transform(original_image).unsqueeze(0).to(self.device)
                with torch.no_grad():
                    output = model(input_tensor)   # shape [1, 8]
                # Convert to CPU numpy (shape [8,])
                logits = output.squeeze(0).cpu().numpy()
                logits_list.append(logits)

            # Average the logits across all folds
            avg_logits = sum(logits_list) / len(logits_list)

            # Convert to torch tensor for convenience
            avg_logits_t = torch.tensor(avg_logits, dtype=torch.float32).unsqueeze(0)  # shape [1,8]
            # Probability from averaged logits
            avg_probs_t = torch.softmax(avg_logits_t, dim=1)
            avg_probs = avg_probs_t.squeeze(0).tolist()

            # Predicted class from the averaged logits
            pred_idx = torch.argmax(avg_logits_t, dim=1).item()
            pred_label = CLASS_LABELS[pred_idx]

            final_predictions[arch_name] = {
                "logits": avg_logits.tolist(),
                "probabilities": avg_probs,
                "class": pred_label
            }

        return final_predictions

    def predict_meta(self, image_path: str, age: float, anatomic_site: str, sex: str):
        """
        1) Averages CNN logits by architecture (via `predict(image_path)`).
        2) Orders them in [EfficientNetB4, EfficientNetB5, EfficientNetB6, Resnext50, Resnet152].
        3) Appends age, anatomic_site, sex.
        4) Feeds feature vector into meta-classifier for final prediction.
        """
        # 1) Get aggregated predictions by architecture
        arch_predictions = self.predict(test_image_path)
                
        # 2) Build feature vector in the specified architecture order
        feature_vector = []
        feature_name = []
        for arch in ARCHITECTURE_ORDER:
            if arch in arch_predictions:
                # Use the averaged logits
                avg_logits = arch_predictions[arch]["logits"]
                # Append them in order
                feature_vector.extend(avg_logits)
                for class_name in CLASS_LABELS:
                    feature_name.append(f"{arch}_{class_name}")
                
        
        # 3) Append metadata        
        feature_vector.append(age)
        feature_name.append("age_approx")
        feature_vector.append(anatomic_site)
        feature_name.append("anatom_site_general")
        feature_vector.append(sex)
        feature_name.append("sex")
        
        # 4) Reshape for sklearn
        X = pd.DataFrame(
            [feature_vector],
            columns = feature_name
        )
        
        # 5) Predict with meta-classifier
        meta_pred = self.meta_model.predict(X)  # final class
        meta_proba = None
        if hasattr(self.meta_model, "predict_proba"):
            meta_proba = self.meta_model.predict_proba(X)[0].tolist()

        return {
            "meta_predicted_label": meta_pred,
            "classes" : self.meta_model.classes_,
            "meta_probabilities": meta_proba
        }

In [176]:
classifier = SkinDiseaseClassifier(
    base_folder="checkpoints", 
    metaclassifier_name = "rf_patient_data",
    device="cuda"
)

Loaded pretrained weights for efficientnet-b4
No loss specified, using default
Model Loaded
Loaded pretrained weights for efficientnet-b4
No loss specified, using default
Model Loaded


Using cache found in /home/cino/.cache/torch/hub/pytorch_vision_v0.9.0


No loss specified, using default
Model Loaded


Using cache found in /home/cino/.cache/torch/hub/pytorch_vision_v0.9.0


No loss specified, using default
Model Loaded
Loaded pretrained weights for efficientnet-b4
No loss specified, using default
Model Loaded


...

No loss specified, using default
Model Loaded
Loaded pretrained weights for efficientnet-b6
No loss specified, using default
Model Loaded
Loaded pretrained weights for efficientnet-b5
No loss specified, using default
Model Loaded


Using cache found in /home/cino/.cache/torch/hub/pytorch_vision_v0.9.0


No loss specified, using default
Model Loaded


Using cache found in /home/cino/.cache/torch/hub/pytorch_vision_v0.9.0


No loss specified, using default
Model Loaded
Loaded pretrained weights for efficientnet-b5
No loss specified, using default
Model Loaded
Loaded pretrained weights for efficientnet-b4
No loss specified, using default
Model Loaded


In [177]:
results = classifier.predict("imagesToTest/IMG_3906.jpg")
results

{'EfficientNetB4': {'logits': [-3.090391159057617,
   10.048822402954102,
   -1.8956981897354126,
   -6.83709716796875,
   1.0210392475128174,
   -5.339770793914795,
   -0.6963459253311157,
   -5.117497444152832],
  'probabilities': [1.9662857084767893e-06,
   0.9998493194580078,
   6.493741693702759e-06,
   4.63951792539774e-08,
   0.00012001016875728965,
   2.0737348904731334e-07,
   2.154602225346025e-05,
   2.589915197859227e-07],
  'class': 'NV'},
 'Resnext50': {'logits': [3.4767792224884033,
   6.28027868270874,
   -5.540708541870117,
   -14.302490234375,
   0.22887957096099854,
   -11.62803840637207,
   -6.831872463226318,
   -10.786775588989258],
  'probabilities': [0.057008299976587296,
   0.940767765045166,
   6.913416200404754e-06,
   1.082677836272694e-09,
   0.002215098822489381,
   1.5703587763482574e-08,
   1.9008487015526043e-06,
   3.6421241134121374e-08],
  'class': 'NV'},
 'Resnet152': {'logits': [1.9155490398406982,
   4.948451042175293,
   -2.397677183151245,
   -1

In [179]:
test_image_path = "imagesToTest/IMG_3906.jpg"
age = 30

#head/neck - Lesions located on the face, scalp, neck, ears, etc.
#upper extremity - Lesions found on arms, hands, and shoulders.
#lower extremity - Lesions found on legs, feet, and hips.
#torso - Lesions found on the chest, abdomen, back, etc.
#palms/soles - Lesions located on the palms of the hands or soles of the feet.
#oral/genital - Lesions located in mucosal regions, such as the mouth or genital areas.

anatomic_site = "torso"
sex = "male"

# Final meta-classifier prediction
meta_result = classifier.predict_meta(
    image_path=test_image_path,
    age=age,
    anatomic_site=anatomic_site,
    sex=sex
)

print("Meta-Classifier Output:")
print("  Predicted Label:", meta_result["meta_predicted_label"])
print("  Classes:", meta_result["classes"])
print("  Probabilities:", meta_result["meta_probabilities"])

Meta-Classifier Output:
  Predicted Label: ['NV']
  Classes: ['AK' 'BCC' 'BKL' 'DF' 'MEL' 'NV' 'SCC' 'VASC']
  Probabilities: [0.000788882653673682, 0.0007411998785662256, 0.02709074486066897, 0.0, 0.0699102664292883, 0.899293978702336, 0.0021749274754668843, 0.0]
